In [ ]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
import matplotlib as mp
# import gaia_tools as gt
import scipy
from scipy.ndimage import gaussian_filter
import astropy.units as u
from astropy.coordinates import SkyCoord
import math
import h5py
import healpy as hp
import pykdgrav3_utils
from pykdgrav3_utils import units
u = units.units(1, 600., verbose=True)

sys.path.append('/mnt/c/gaiaenv/New Py/My_thesis/Modules')
from mock_wake import generate_mock_wake
from rotation_funcs import rotate, angle_finder, rz, ry, rx

In [ ]:
file = '/mnt/c/Data/Praktika_data/dm_sim.00001.0'

def load_snap_file(path, part_type='PartType1', is_print = False):

    snap_file = h5py.File(path, 'r')
    part_data = snap_file[part_type]

    if(is_print):
        print(f'Loading snapshot: {path.split("/")[-1]}')
        print(f'Selected species: {part_type}')
        print(f'Snap file keys: {snap_file.keys()}')
        print(f'Part type keys: {part_data.keys()}')

    return part_data

In [ ]:
snap_stars = load_snap_file(file, part_type='PartType4', is_print=True)
star_coord = snap_stars['Coordinates'][:]*u.dKpcUnit

In [ ]:
## Find LMC velocity when it is at 70 kpc distance. Also find its current position.

orbitfile = '/mnt/c/Data/Praktika_data/trajlmc.txt'
orbit_full = pd.read_csv(orbitfile, delimiter = ' ')
orbit = orbit_full.loc[np.where(orbit_full['time'] < 0.01)]
d_orbit = np.sqrt(orbit['x']**2 + orbit['y']**2 + orbit['z']**2)
loc70 = np.isclose(d_orbit, 70.0, 0.01)
# 70 - d_orbit[loc70]
v70 = orbit['Vx'][loc70].values[0], orbit['Vy'][loc70].values[0], orbit['Vz'][loc70].values[0]

LMC_today = np.isclose(orbit['time'], 0.0, 0.001)
LMC_loc_today = orbit[['x', 'y', 'z']][LMC_today].values[0]


In [ ]:
## Rotate simulation

z_angle, y_angle = angle_finder(v70)

R = rz(z_angle) @ ry(y_angle)
star_coord_rot = R @ star_coord.T
star_coord_rot = star_coord_rot.T


In [ ]:
plt.hist2d(star_coord[:,0][::500], star_coord[:,1][::500], bins=50, cmap='viridis')